In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from utils.datasets import create_dataloader
from utils.midog_dataset import MidogDataset

/home/ammeling/projects/yolov7/utils/midog_dataset.py:18: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
ds = MidogDataset(
    split='train',
    img_dir='/data/patho/MIDOG2/',
    dataset='/home/ammeling/projects/Bhattacharyya/annotations/MIDOG2022_training.csv',
    augment=True,
    box_format='cxcy',
    num_samples=1024,
    patch_size=1024
)

Initializing slide objects:   0%|          | 0/283 [00:00<?, ?it/s]

Initializing slide objects: 100%|██████████| 283/283 [00:00<00:00, 694.65it/s]


In [4]:
sample = ds.samples[2]

# extract file and coords
file, coords = sample['file'], sample['coords']

# get slide object
slide = ds.slide_objects[file]

# load image and boxes
img = slide.load_image(coords)
targets = slide.load_labels(coords, label=0)  # load only MF labels
labels = targets[:, 0]
boxes = targets[:, 1:]

boxes

array([[          0,           0,           0,           0]], dtype=float32)

In [5]:
ds[2][1].shape

torch.Size([1, 6])

In [5]:
from torch.utils.data import DataLoader

dl = DataLoader(ds, batch_size=4, collate_fn=ds.collate_fn)

In [6]:
for batch in dl:
    images, targets, files, shapes = batch 
    break 

In [7]:
images[0]

tensor([[[0.75294, 0.77255, 0.76863,  ..., 0.82353, 0.81569, 0.78431],
         [0.69412, 0.73725, 0.76078,  ..., 0.84314, 0.83922, 0.80784],
         [0.69020, 0.74118, 0.78824,  ..., 0.84706, 0.84314, 0.84314],
         ...,
         [0.73725, 0.70588, 0.68627,  ..., 0.80392, 0.80392, 0.83922],
         [0.78431, 0.73333, 0.69412,  ..., 0.79216, 0.79608, 0.84314],
         [0.77647, 0.69412, 0.61961,  ..., 0.80000, 0.80000, 0.83137]],

        [[0.48627, 0.49412, 0.48235,  ..., 0.61961, 0.58431, 0.56078],
         [0.42745, 0.47059, 0.48235,  ..., 0.65098, 0.61569, 0.58039],
         [0.40784, 0.43922, 0.47843,  ..., 0.69804, 0.65490, 0.63137],
         ...,
         [0.49804, 0.46667, 0.45882,  ..., 0.48235, 0.48235, 0.51373],
         [0.54510, 0.49020, 0.44706,  ..., 0.47451, 0.47843, 0.51765],
         [0.54118, 0.46275, 0.38824,  ..., 0.49020, 0.48627, 0.52157]],

        [[0.69020, 0.71765, 0.73725,  ..., 0.84706, 0.82745, 0.75294],
         [0.63137, 0.69020, 0.74118,  ..., 0.

In [8]:
import torch
torch.min(images[0]), torch.max(images[0])

(tensor(0.08627), tensor(0.98431))

In [10]:
from models.yolo import Model
import yaml 

device = 'cuda:0'
hyp = yaml.load(open('data/hyp.scratch.p5.yaml', 'rb'), Loader=yaml.SafeLoader)  # load hyps'data/hyp.scratch.p5.yaml'

model = Model(
    cfg='cfg/training/yolov7-midog.yaml',
    ch=3,
    nc=1
).to(device)
model.hyp = hyp
model.gr = 1.0

INFO:models.yolo:
                 from  n    params  module                                  arguments                     
INFO:models.yolo:  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
INFO:models.yolo:  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
INFO:models.yolo:  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
INFO:models.yolo:  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
INFO:models.yolo:  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
INFO:models.yolo:  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
INFO:models.yolo:  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
INFO:mo

In [11]:
from utils.loss import ComputeLoss

compute_loss = ComputeLoss(model)


In [12]:
targets

tensor([[0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000],
        [1.00000, 0.00000, 0.27832, 0.24707, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.25781, 0.21973, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.12988, 0.12598, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.13379, 0.78613, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.21484, 0.83203, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.74121, 0.45215, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.86621, 0.43848, 0.04883, 0.04883],
        [2.00000, 0.00000, 0.05859, 0.51758, 0.04883, 0.04883],
        [3.00000, 0.00000, 0.75781, 0.60742, 0.04883, 0.04883]])

In [13]:
images.shape, targets.shape

(torch.Size([4, 3, 1024, 1024]), torch.Size([10, 6]))

In [14]:
targets

tensor([[0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000],
        [1.00000, 0.00000, 0.27832, 0.24707, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.25781, 0.21973, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.12988, 0.12598, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.13379, 0.78613, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.21484, 0.83203, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.74121, 0.45215, 0.04883, 0.04883],
        [1.00000, 0.00000, 0.86621, 0.43848, 0.04883, 0.04883],
        [2.00000, 0.00000, 0.05859, 0.51758, 0.04883, 0.04883],
        [3.00000, 0.00000, 0.75781, 0.60742, 0.04883, 0.04883]])

In [15]:
pred = model(images.to(device))

In [16]:
loss, loss_items = compute_loss(pred, targets.to(device))

In [17]:
loss, loss_items

(tensor([0.55835], device='cuda:0', grad_fn=<MulBackward0>),
 tensor([0.11306, 0.02653, 0.00000, 0.13959], device='cuda:0'))

In [24]:
model

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (4): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), 